In [174]:
## ABRIMOS EL DATASET

import pickle
import time
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Cargar el dataset 
# ¡Cambiar según el que se quiera usar!
####
#with open("dataset_dhl_1.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_2.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_3.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_4.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_5.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_6.pkl", "rb") as f:
#      data = pickle.load(f)
#with open("dataset_dhl_7.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_8.pkl", "rb") as f:
#    data = pickle.load(f)
#with open("dataset_dhl_9.pkl", "rb") as f:
#    data = pickle.load(f)  
with open("dataset_dhl_10.pkl", "rb") as f:
   data = pickle.load(f)      



In [175]:

## AÑADIMOS LAS COORDENADAS ##

# Metemos las coordenadas que ya conocemos (las que no podemos calcular con geopy)
coordenadas_fijas = {
    "Atlante - Atlante - Nieves Sant Esteve": (41.504070650751906, 1.883253906745552),
    "Atlante - Nieves Zona Franca":( 41.33194049366559, 2.134952349708804),
    "IONITY Montcada i Reixac": (41.491422605263075, 2.1823508377517684),
    "IONITY Barcera del Valles": (41.51182358577195, 2.1345502627626303),
    "Ctra. Molins de Rei, 251, Km. 8, 08191 Rubí, Barcelona": (41.46930679260032, 2.029808746545829),
    "Carrer Sakura, 16B, NAVE 16A, 08272 Sant Fruitós de Bages, Barcelona": (41.77803590470732, 1.867141253552206),
    "Avinguda Can Montcau, 1-3-5, 08186 Lliçà d'Amunt, Barcelona": (41.60289038509304, 2.2569771240231096),
    "Carrer d'Anselm Clavé, 62 AB, 08186 Lliçà d'Amunt, Barcelona":(41.60827, 2.239330),
    "Carrer Ferralla, 24, 08755 Castellbisbal, Barcelona":(41.44667, 1.983210),
    "Pol. Ind. Can Bosch, Mod, 2, 3/4, 08739, Barcelona": (41.42833, 1.819261),
    "Carrer Ferralla 14":(41.45037, 1.979796),
    "POL.IND. LES MINITES, PARCELA2-3, CIM DEL VALLÉS": (41.53960, 2.183749),
    "Carrer d\'Edison, 16":(41.5573862,2.2609581),
    "Carrer Manganès 2" :(41.4680828,1.9515915),
    "DHL FREIGHT SPAIN S.L, C/ Atlantic 109-111  (ZAL)": (41.548266507563135, 2.1687455645705684)
}

# Inicializar geolocalizador 
geolocator = Nominatim(user_agent="geoapi")

# Añadir coordenadas 
for d in data:
    direccion = d.get("direccion", "").strip()
    nombre = d.get("nombre", "").strip()

    # Caso 1: coordenadas conocidas
    if direccion in coordenadas_fijas:
        d["coordenadas"] = coordenadas_fijas[direccion]

    elif nombre in coordenadas_fijas:
        d["coordenadas"] = coordenadas_fijas[nombre]

    # Caso 2: intentar buscar con geopy
    elif direccion:
        try:
            location = geolocator.geocode(direccion)
            if location:
                d["coordenadas"] = (location.latitude, location.longitude)
            else:
                d["coordenadas"] = None
                print(f"No se encontró la dirección: {nombre}")
        except Exception as e:
            print(f"Error al buscar {nombre}: {e}")
            d["coordenadas"] = None
        time.sleep(1)  # Evita bloqueo por exceso de peticiones

    else:
        d["coordenadas"] = None
        print(f" No hay dirección para: {nombre}")





In [176]:
## MATRIZ DE DISTANCIAS 
n = len(data)
distancias = [[0]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i != j and data[i]["coordenadas"] and data[j]["coordenadas"]:
            dist = geodesic(data[i]["coordenadas"], data[j]["coordenadas"]).km
            distancias[i][j] = round(dist, 2)
        else:
            distancias[i][j] = None if i != j else 0


#Mostrar matriz
for fila in distancias:
    print(fila)

    

[0, 16.64, 36.87, 24.07, 18.17, 14.54, 5.92, 9.94, 12.98, 17.61, 12.83]
[16.64, 0, 52.74, 36.68, 21.48, 27.87, 11.46, 23.03, 22.88, 8.76, 23.96]
[36.87, 52.74, 0, 37.84, 39.04, 35.81, 41.3, 37.02, 41.27, 54.35, 30.46]
[24.07, 36.68, 37.84, 0, 41.36, 9.54, 29.41, 14.37, 13.86, 31.77, 33.05]
[18.17, 21.48, 39.04, 41.36, 0, 32.08, 15.3, 27.92, 31.13, 28.49, 9.97]
[14.54, 27.87, 35.81, 9.54, 32.08, 0, 19.9, 4.95, 6.41, 24.19, 24.33]
[5.92, 11.46, 41.3, 29.41, 15.3, 19.9, 0, 15.04, 17.02, 14.83, 13.6]
[9.94, 23.03, 37.02, 14.37, 27.92, 4.95, 15.04, 0, 4.59, 19.98, 21.0]
[12.98, 22.88, 41.27, 13.86, 31.13, 6.41, 17.02, 4.59, 0, 18.15, 25.01]
[17.61, 8.76, 54.35, 31.77, 28.49, 24.19, 14.83, 19.98, 18.15, 0, 28.43]
[12.83, 23.96, 30.46, 33.05, 9.97, 24.33, 13.6, 21.0, 25.01, 28.43, 0]


In [177]:
## MATRIZ DE TIEMPOS (d):
# Vamos a suponer que el camion va a una velocidad media de 70km/h
tiempos = [[0]*n for _ in range(n)]
velocidad=70
for i in range(len(distancias)):
    for j in range(len(distancias)):
        tiempos[i][j]=distancias[i][j]/70*60 # en minutos
for filas in tiempos:
    print(filas)  

[0.0, 14.262857142857143, 31.60285714285714, 20.63142857142857, 15.574285714285717, 12.462857142857143, 5.074285714285715, 8.52, 11.125714285714286, 15.094285714285714, 10.997142857142856]
[14.262857142857143, 0.0, 45.205714285714286, 31.44, 18.411428571428573, 23.888571428571428, 9.822857142857144, 19.740000000000002, 19.611428571428572, 7.508571428571428, 20.537142857142857]
[31.60285714285714, 45.205714285714286, 0.0, 32.434285714285714, 33.462857142857146, 30.694285714285712, 35.4, 31.731428571428573, 35.37428571428572, 46.58571428571429, 26.10857142857143]
[20.63142857142857, 31.44, 32.434285714285714, 0.0, 35.45142857142857, 8.177142857142856, 25.208571428571428, 12.317142857142857, 11.879999999999999, 27.23142857142857, 28.328571428571426]
[15.574285714285717, 18.411428571428573, 33.462857142857146, 35.45142857142857, 0.0, 27.497142857142855, 13.114285714285716, 23.931428571428572, 26.68285714285714, 24.419999999999998, 8.545714285714286]
[12.462857142857143, 23.888571428571428,

In [ ]:
## VECTOR DE PRECIO CARGA (w)
import re
precio_ionity = 0.61
precio_atlante = 0.64
kwh = 300


for d in data:
    if re.search("IONITY",d["nombre"]):
        d["w"]=kwh*precio_ionity
    elif re.search("Atlante",d["nombre"]):
         d["w"]=kwh*precio_atlante
    else:
         d["w"]=0

              


0
0
0
0
0
0
183.0
183.0
183.0
192.0
192.0


In [179]:
# MATRIZ DE COSTES (cij)
# cij=dist*precio
precio=0.4
c=[[0]*len(data) for _ in range(len(data))]
for i in range(len(data)):
    for j in range(len(data)):
        c[i][j]=distancias[i][j]*0.4
for filas in c:
    print(filas)
   


[0.0, 6.656000000000001, 14.748, 9.628, 7.268000000000001, 5.816, 2.368, 3.976, 5.192, 7.0440000000000005, 5.132000000000001]
[6.656000000000001, 0.0, 21.096000000000004, 14.672, 8.592, 11.148000000000001, 4.5840000000000005, 9.212000000000002, 9.152, 3.504, 9.584000000000001]
[14.748, 21.096000000000004, 0.0, 15.136000000000003, 15.616, 14.324000000000002, 16.52, 14.808000000000002, 16.508000000000003, 21.740000000000002, 12.184000000000001]
[9.628, 14.672, 15.136000000000003, 0.0, 16.544, 3.816, 11.764000000000001, 5.748, 5.5440000000000005, 12.708, 13.219999999999999]
[7.268000000000001, 8.592, 15.616, 16.544, 0.0, 12.832, 6.120000000000001, 11.168000000000001, 12.452, 11.396, 3.9880000000000004]
[5.816, 11.148000000000001, 14.324000000000002, 3.816, 12.832, 0.0, 7.96, 1.9800000000000002, 2.564, 9.676000000000002, 9.732]
[2.368, 4.5840000000000005, 16.52, 11.764000000000001, 6.120000000000001, 7.96, 0.0, 6.016, 6.808, 5.932, 5.44]
[3.976, 9.212000000000002, 14.808000000000002, 5.748

In [180]:
## CREAR NUEVO DATA SET
dataset_actualizado = {
    "data": data,
    "distancias": distancias, 
    "tiempos": tiempos,
    "costes":c
}

#with open("dataset_1_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_2_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_3_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_4_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_5_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_6_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_7_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_8_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
#with open("dataset_9_actualizado.pkl", "wb") as f:
#    pickle.dump(dataset_actualizado, f)
with open("dataset_10_actualizado.pkl", "wb") as f:
    pickle.dump(dataset_actualizado, f)
